In [167]:
%pip install rdkit
%pip install pysmiles
%pip install networkx
%pip install spektral
%pip install tensorflow
%pip install keras
%pip install matplotlib
%pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.

In [46]:
from pysmiles import read_smiles
import networkx as nx
from spektral.data import Dataset, DisjointLoader, Graph
import numpy as np
import random
import pandas as pd

In [168]:
pp = pd.read_csv('../data/test.csv')
pp = pp[["name", "p_np", "smile"]]
pp.head()

,name,p_np,smile
0,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


In [170]:
tempList = []

##hashing here sucks, should find a better way

for index, row in pp.iterrows():
    smiles = row["smile"]
    mol = read_smiles(smiles)
    nx.convert_node_labels_to_integers(mol)
    nodevec = list(mol.nodes(data='element'))

    ##change this later to the actual element value, just tryna get some loss and the nn actually working
    nodevec = [1 for x in nodevec]
    adj = nx.to_pandas_adjacency(mol).to_numpy().astype(int)


    ##change this later to the actual element value, just tryna get some loss and the nn actually working
    g = Graph(a = adj, y = int(row["p_np"]))
    tempList.append(g)


Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@]" contains stereochemical information that will be discarded.
Atom "[C@]" contains stereochemical information that will be discarded.
E/Z stereochemical information, which is specified by "\", will be discarded
E/Z stereochemical information, which is specified by "/", will be discarded
Atom "[C@@H]" contains stereochemical inf

In [171]:
pp["graph"] = tempList

graphs = pp["graph"]
graphs


0       Graph(n_nodes=20, n_node_features=None, n_edge...
1       Graph(n_nodes=23, n_node_features=None, n_edge...
2       Graph(n_nodes=26, n_node_features=None, n_edge...
3       Graph(n_nodes=21, n_node_features=None, n_edge...
4       Graph(n_nodes=29, n_node_features=None, n_edge...
                              ...                        
2032    Graph(n_nodes=17, n_node_features=None, n_edge...
2033    Graph(n_nodes=27, n_node_features=None, n_edge...
2034    Graph(n_nodes=24, n_node_features=None, n_edge...
2035    Graph(n_nodes=28, n_node_features=None, n_edge...
2036    Graph(n_nodes=21, n_node_features=None, n_edge...
Name: graph, Length: 2037, dtype: object

In [172]:
class MyDataset(Dataset):
    """
    load our data
    """
    def __init__(self, graphs, **kwargs):
        self.graphs = graphs
        super().__init__(**kwargs)
    

        
    def read(self):
        # We must return a list of Graph objects
        return self.graphs


In [151]:
dataset = MyDataset(graphs = graphs)

In [192]:
dataset.filter(lambda g: g.n_nodes < 45)

In [154]:
max_degree = dataset.map(lambda g: g.a.sum(-1).max(), reduce=max)
max_degree

4

In [188]:
from spektral.transforms import Degree

dataset.apply(Degree(max_degree))

Graph(n_nodes=20, n_node_features=None, n_edge_features=None, n_labels=1)

In [165]:
from spektral.transforms import GCNFilter

dataset.apply(GCNFilter())

AttributeError: 'NoneType' object has no attribute 'apply'

In [193]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense1 = Dense(n_hidden, 'relu')
        self.dense2 = Dense(n_hidden, 'relu')
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense1(out)
        out = self.dropout(out)
        out = self.dense2(out)
        out = self.dropout(out)
        out = self.dense(out)

        return out


In [194]:
from keras.optimizers import Adam

model = MyFirstGNN(10000, dataset.n_labels)
opt = Adam(learning_rate=.01)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [196]:
from spektral.data import BatchLoader

np.random.shuffle(dataset)
split = int(0.8 * len(dataset))
data_tr, data_te = dataset[:split], dataset[split:]

loader = BatchLoader(data_tr, batch_size=32)

/var/folders/xf/mwvkzqj51j93w8hrtltgvgn40000gn/T/ipykernel_91301/873634478.py:3: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset)


In [197]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=30)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/30


2023-12-04 14:36:10.577685: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


11/23 [=============>................] - ETA: 11s - loss: 11510.6367 - accuracy: 0.5227

In [161]:
loader = BatchLoader(data_te, batch_size=32)
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))


2023-12-04 14:10:34.326261: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


6/6 [==============================] - 1s 62ms/step - loss: 3.4871 - accuracy: 0.5667
Test loss: [3.487107753753662, 0.5666666626930237]


In [82]:
data_te[0].a

array([[0.5       , 0.35355339, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.35355339, 0.25      , 0.25      , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.35355339],
       [0.        , 0.25      , 0.25      , 0.25      , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.35355339,
        0.        ],
       [0.        , 0.        , 0.25      , 0.25      , 0.28867513,
        0.        , 0.        , 0.        , 0.35355339, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.28867513, 0.33333333,
        0.28867513, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.28867513,
        0.25      , 0.35355339, 0.35355339, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.